# **High Value Customer Identification - Notebook A** 

Context
Typically e-commerce datasets are proprietary and consequently hard to find among publicly available data. However, The UCI Machine Learning Repository has made this dataset containing actual transactions from 2010 and 2011. The dataset is maintained on their site, where it can be found by the title "Online Retail".

Content
"This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers."

**Tópicos neste notebook:**

1 - Data Description

2 - Feature Engineering

3 - Filtragem de Variáveis

# Planejamento da Solução (IOT)

## Input - Entrada

1. Problema de negócio: selecionar os mais valiosos clientes para formar o programa de fidelidade **"INSIDERS"**
2. Conjunto de dados com as vendas de produtos durante 1 ano ( Dez 2010 to Dez 2011)

## Output - Saída

1. A indicação de pessoas para fazer parte do programa de fidelidade **"INSIDERS"**.

2. Relatório com as respostas para as seguintes perguntas:
    - Quem são as pessoas elegíveis para participar do programa de Insiders ?
    - Quantos clientes farão parte do grupo?
    - Quais as principais características desses clientes ?
    - Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
    - Qual a expectativa de faturamento desse grupo para os próximos meses ?
    - Quais as condições para uma pessoa ser elegível ao Insiders ?
    - Quais as condições para uma pessoa ser removida do Insiders ?
    - Qual a garantia que o programa Insiders é melhor que o restante da base ?
    - Quais ações o time de marketing pode realizar para aumentar o faturamento?

## Tasks - Processo

1. **Quem são as pessoas elegíveis para participar do programa de Insiders ?**
    - O que é ser elegível ? O que é um cliente "valioso" para a empresa ?
        - Faturamento:
            - Alto Ticket Médio
            - Alto LTV
            - Baixa Recência ou Alta Frequência ( tempo entre as compras )
            - Alto Basket Size ( quantidade média de produtos comprados )
            - Baixa probabilidade de Churn
            - Previsão alta de LTV
            - Alta propensão de compra

        - Custo:
            - Baixo número de devoluções

        - Experiência:  
            - Média alta de avaliações
            
            
2. **Quantos clientes farão parte do grupo?**
    - Número de clientes
    - % em relação ao total de clients
    
    
3. **Quais as principais características desses clientes ?**
    - Escrever os principais atributos dos clientes
        - Idade
        - País
        - Salário
        
    - Escrever os principais comportamentos de compra dos clients ( métricas de negócio )
        - Vide acima
    
    
4. **Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?**
    - Calcular o faturamento total da empresa durante o ano.
    - Calcular o faturamento (%) apenas do cluster Insiders.
    
    
5. **Qual a expectativa de faturamento desse grupo para os próximos meses ?**
    - Cálculo do LTV do grupo Insiders
    - Séries Temporais ( ARMA, ARIMA, HoltWinter, etc )
    

6. **Quais as condições para uma pessoa ser elegível ao Insiders ?**
    - Qual o período de avaliação ?
    - O "desempenho" do cliente está próximo da média do cluster Insiders. 
    
    
7. **Quais as condições para uma pessoa ser removida do Insiders ?**
    - O "desempenho" do cliente não está mais próximo da média do cluster Insiders. 
    
    
8. **Qual a garantia que o programa Insiders é melhor que o restante da base ?**
    - Teste de Hipóteses
    - Teste A/B
    
    
9. **Quais ações o time de marketing pode realizar para aumentar o faturamento?**
    - Descontos
    - Preferências de escolha
    - Produtos exclusivos

# 0.0 Imports

In [1]:
import pandas as pd
import numpy as np
import inflection
import math
import re

from unidecode               import unidecode
from IPython.display         import Image, display

## 0.1 Helper Function

In [2]:
# Função para separar letras e números (filtrando dataset - produtos comprados (b) e cancelados(c))
def separar_letras_e_numeros(codigo):
    padrao = re.match(r'([A-Za-z]*)(\d*)', codigo)
    
    if padrao:
        letras = padrao.group(1)  # Converta para maiúsculas se necessário
        numeros = padrao.group(2)
        return letras, numeros
    else:
        return None, None

# -------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------

# Função para encontrar a média em dias, entre as últimas 3 compras do cliente
def calculo_rca_media_3(dates_list, max_date):

    # Colocando lista em ordem descrescente 
    sorted_dt_list = sorted(dates_list, reverse=True)
    
    # Encontrado o intervalo de tempo entre compras, em dias
    if len(dates_list) == 1:
        c1 = (max_date - sorted_dt_list[0]).days
        return c1
    
    if len(dates_list) == 2:
        c1 = (max_date - sorted_dt_list[0]).days
        c2 = (sorted_dt_list[0] - sorted_dt_list[1]).days
        return (c1+c2) / 2

    if len(dates_list) >= 3:
        c1 = (max_date - sorted_dt_list[0]).days
        c2 = (sorted_dt_list[0] - sorted_dt_list[1]).days
        c3 = (sorted_dt_list[1] - sorted_dt_list[2]).days
        return (c1+c2+c3) / 3

# -------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------

# Função para encontrar a média, em dias, entre a diferença (diff) de todas as datas de compras do cliente
def calculo_rca_media_total(dates_list, max_date):

    # Colocando as datas em ordem decrescente
    sorted_dt_list = sorted(dates_list, reverse=True)

    # Inserindo a última data do dataset em primeiro lugar da lista ordenada
    sorted_dt_list.insert(0, max_date)

    # Calculando a diferença entre cada data
    date_diff_list = [(sorted_dt_list[i] - sorted_dt_list[i - 1]).days for i in range(1, len(sorted_dt_list))]
    
    # Calculando a média de todos os intervalos
    media_rca_total = abs( sum(date_diff_list) / len(date_diff_list) )

    return media_rca_total



# -------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------

# Configurações Gerais
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

## 0.2 Loading data

In [3]:
# Dataset treino
df_raw = pd.read_csv('../datasets/raw_datasets/data.csv', low_memory = False, encoding='ISO-8859-1')

# Removendo coluna vazia, devido ao csv ter vírgulas no final de cada linha
df_raw = df_raw.drop(['Unnamed: 8'], axis=1)

# **1.0 Data Description**

In [4]:
df1 = df_raw.copy()

In [5]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.00,United Kingdom


## 1.1 Rename Columns

In [6]:
# Renomeando as colunas usando a função underscore do inflection
df1.columns = [inflection.underscore(col) for col in df1.columns]
df1.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

## 1.2 Data Dimension

In [7]:
# Printando linhas/colunas
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

Number of Rows: 541909
Number of Cols: 8


## 1.3 Data Types

In [8]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4. Check NA

In [9]:
'''
Existem 3 opções para tratar os NAs

1- Descartar linhas com valores ausentes (NA):
    Vantagem: rápido e fácil.
    Desvantagem (significativa): Pode levar à perda de informações importantes e prejudicar a performance do modelo se o conjunto de dados for pequeno.

2- Utilizar algoritmos de Machine Learning:
    Existem métodos para preencher valores ausentes (NA) que se baseiam no comportamento das colunas.
    Podemos utilizar métodos estatísticos como a média ou mediana, ou algorimos de Machine Learning para prever valores ausentes.
    Essa técnica é útil quando não se tem informações de negócio.

3- Entender a lógica de negócio:
    Compreendendo a lógica de negócio, é possível identificar o motivo dos valores ausentes e estabelecer regras para preenchê-los.  

Utilizarei a opção 3 para preencher os valores faltantes (NA) no próximo tópico.
'''

df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Fillout NA

In [10]:
# Preenchendo os NaN com lógica de negócio

# description
df1['description'] = df1['description'].fillna('no_description')

# customer_id
df1 = df1.dropna(subset=['customer_id'])

# Verificação da remoção dos NaN
df1.isna().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64

In [11]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.00,United Kingdom


## 1.6 Change Types

In [12]:
# Verificando os tipos de dados novamente após as alterações feitas no tópico 1.5, por boa prática.  
df1.dtypes

# Covertendo
df1['customer_id'] = df1['customer_id'].astype(int)
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])
df1.dtypes

C:\Users\andre\AppData\Local\Temp\ipykernel_3592\946814893.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1['invoice_date'] = pd.to_datetime(df1['invoice_date'])


invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id              int32
country                 object
dtype: object

O conjunto de dados inclui as seguintes informações:

| Coluna                     | Descrição                                           |
|----------------------------|-----------------------------------------------------|
| Invoice Number             | Identificador único de cada transação.              |
| Stock Code Product         | Código do item.                                    |
| Description Product        | Nome do item.                                      |
| Quantity                   | A quantidade de cada item comprado por transação.  |
| Invoice Date               | O dia em que a transação ocorreu.                   |
| Unit Price                 | Preço do produto por unidade.                      |
| Customer ID                | Identificador único do cliente.                     |
| Country                    | O nome do país onde o cliente reside.               |
| invoice_date_formatted     | Data formatada (dia/mês/ano).                      |
| month                      | Número do mês.                                     |


## 1.7 Minority Changes (map data)

In [13]:
df1 = df1.drop_duplicates()

# **2.0 Feature Engineering**

In [14]:
df2 = df1.copy()

## **2.1 MindMap Hypotheses**

<p>O Mindmap de Hipóteses nos ajuda a criar hipóteses sobre o fenômeno estudado. Tais hipóteses geradas serão investigadas na EDA (Análise Exploratória de Dados).</p>
<p>Na imagem abaixo encontra-se o Mindmap de hipóteses, onde ao centro, o 'Preço do Veículo' é o fenômeno a ser investigado e está rodeado por suas entidades 'Vendedor', 'Veículo' e 'Documentação do Veículo', seguidas por seus atributos.

In [15]:
# Image('../images/mindmap_.png')

NOTA: No cotidiano de uma empresa, as etapas iniciais são compostas por:

1- Investigação individual: Nesta fase, eu, como profissional responsável pelo projeto, realizo investigações e análises preliminares, criando incialmente meu próprio mindmap.

2- Após a investigação inicial, compartilho minhas descobertas com a área de negócio, onde os stakeholders avaliam e contribuem com suas perspectivas e ideias.

3- Com base nas ideias reunidas tanto por mim quanto pela área de negócio, a equipe técnica e de análise de dados entra em ação. Juntos, verificamos quais dados estão disponíveis no banco de dados para implementar as ideias discutidas anteriormente.

Obs.: Os atributos com asterisco não estão presentes no dataset, mas é interessante investigar, pois pode contribuir para entender mais sobre o problema de negócio e talvez melhorar a performance do modelo de machine learning.

## **2.2 Hypotheses Creation**

### 2.2.1 Hipóteses do Veículo

**1.** Carros mais antigos possuem um preço menor.

**2.** Carros com maior valor de hodômetro possuem valores menores.

**3.** Carros na cor branca possuem preços maiores.

**4.** Carros com cambio manual e com valores maiores de hodometro, possuem preços menores.

### 2.2.2 Hipóteses da Documentação

**1.** Carros com garantia de fábrica possuem preços maiores.

**2.** Carros com revisões feitas em dia e nas concessionárias, possuem preços maiores.


### 2.2.3 Hipóteses do Vendedor

**1.** Carros vendidos por pessoa física possuem preços menores.

**2.** Carros da região Sudeste possuem preços maiores em média.

**3.** Vendedores que aceitam troca possuem carros com preços menores.

**4.** Carros populares de baixo padrão são mais vendidos por pessoas físicas.

## **2.3 Lista Final de Hipóteses**

**1.** Carros mais antigos possuem um preço menor.

**2.** Carros com maior valor de hodômetro possuem valores menores.

**3.** Carros na cor branca possuem preços maiores.

**4.** Carros com cambio manual e com valores maiores de hodometro, possuem preços menores.

**5.** Carros com garantia de fábrica possuem preços maiores.

**6.** Carros com revisões feitas em dia e nas concessionárias, possuem preços maiores.

**7.** Carros vendidos por pessoa física possuem preços menores.

**8.** Carros da região Sudeste possuem preços maiores em média.

**9.** Vendedores que aceitam troca possuem carros com preços menores.

**10.** Carros populares de baixo padrão são mais vendidos por pessoas físicas.

## **2.4 Feature Engineering**

In [16]:
df2.describe().T

,count,mean,min,25%,50%,75%,max,std
quantity,401603.00,12.18,-80995.00,2.00,5.00,12.00,80995.00,250.28
invoice_date,401603,2017-07-07 22:53:53.288595712,2016-11-29 00:00:00,2017-04-04 00:00:00,2017-07-27 00:00:00,2017-10-18 00:00:00,2017-12-07 00:00:00,NaN
unit_price,401603.00,3.47,0.00,1.25,1.95,3.75,38970.00,69.76
customer_id,401603.00,15281.16,12346.00,13939.00,15145.00,16784.00,18287.00,1714.01


In [17]:
df2.head(1)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,17850,United Kingdom


### 2.4.1 Faturamento

In [18]:
# Faturamento por linha 
df2['faturamento'] = df2['quantity'] * df2['unit_price']

### 2.4.2 Separando dataset - Comprados(b) Cancelados(c)

In [19]:
# Aplicar a função a cada valor na coluna 'codigo'
df2[['invoice_letter', 'invoce_number']] = df2['invoice_no'].apply(lambda x: pd.Series(separar_letras_e_numeros(x)))

# Verificando se todos os valores da coluna quantidade são negativos.
df2[(df2['invoice_letter'] == 'C') & (df2['quantity'] > 0)].shape[0]

# Filtrando dataset
# Produtos cancelados 
df2c = df2[df2['invoice_letter'] == 'C']

# Produtos comprados 
df2b = df2[df2['invoice_letter'] != 'C']

prct_cancel = round(df2c.shape[0] / df2b.shape[0], 2) * 100
print(f'Os cancelamentos representam {prct_cancel}% do total de vendas')

Os cancelamentos representam 2.0% do total de vendas


### 2.4.3 Ticket médio, Valor total das compras, Total de transações (Todos por cliente)

In [20]:
# Soma de compras por cliente total de transações por cliente
fatur_cliente = df2b.groupby('customer_id').agg({ 'faturamento': 'sum', 'invoice_no':'nunique'}).reset_index()
# nunique aqui, pois os invoice_no se repetem devido cada linha do dataset ser um produto, ou seja, ele pode ter feito uma compra mas com vários produtos
# então aparecerá vários invoice_no repetidos.

# Renomeando colunas
fatur_cliente.columns = ['customer_id', 'total_valor', 'total_transacoes']

# Encontrando ticket médio
fatur_cliente['ticket_medio'] = fatur_cliente['total_valor'] / fatur_cliente['total_transacoes']
fatur_cliente

,customer_id,total_valor,total_transacoes,ticket_medio
0,12346,77183.60,1,77183.60
1,12347,4310.00,7,615.71
2,12348,1797.24,4,449.31
3,12349,1757.55,1,1757.55
4,12350,334.40,1,334.40
...,...,...,...,...
4334,18280,180.60,1,180.60
4335,18281,80.82,1,80.82
4336,18282,178.05,2,89.03
4337,18283,2045.53,16,127.85


### 2.4.4 Recencia (dias)

RECÊNCIA em dias, da última compra:

In [21]:
# RECÊNCIA em dias, da última compra:

# Data da última compra de cada cliente
rca_ultima_compra = df2b.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
# Renomeando colunas
rca_ultima_compra.rename(columns={'invoice_date': 'ultima_compra'}, inplace=True)

# Data maxima do dataset df2b inteiro
rca_ultima_compra['data_max'] = df2b['invoice_date'].max()
# Encontrando em dias, a diferença do ultimo dia de compra do cliente com a data max do dataset
rca_ultima_compra['recencia_ultima_compra'] = (rca_ultima_compra['data_max'] - rca_ultima_compra['ultima_compra']).dt.days
rca_ultima_compra

,customer_id,ultima_compra,data_max,recencia_ultima_compra
0,12346,2017-01-16,2017-12-07,325
1,12347,2017-12-05,2017-12-07,2
2,12348,2017-09-23,2017-12-07,75
3,12349,2017-11-19,2017-12-07,18
4,12350,2017-01-31,2017-12-07,310
...,...,...,...,...
4334,18280,2017-03-05,2017-12-07,277
4335,18281,2017-06-10,2017-12-07,180
4336,18282,2017-11-30,2017-12-07,7
4337,18283,2017-12-04,2017-12-07,3


RECÊNCIA média, em dias, das 3 últimas compras:

In [22]:
# RECÊNCIA média, em dias, das 3 últimas compras:

# Agrupando todos os chamados de cada cliente e utilizando o set para pegar apenas uma data unica, estava
# repetindo as datas, pois cada linha do dataset é um produto, se o cliente comprou mais de um produto no 
# mesmo dia resultava em várias datas iguais na coluna invoice_date com código de transação invoice_no repetido
rca_media_3 = df2b.groupby('customer_id')['invoice_date'].agg(lambda x: list(set(x))).reset_index()
rca_media_3.columns = ['customer_id', 'unique_invoice_dates']
rca_media_3

# Data maxima do dataset df2b inteiro
max_date = df2b['invoice_date'].max()
# Encontrando em dias, a diferença dos 3 últimos dias de compra do cliente com a data max do dataset
rca_media_3['recencia_media_3'] = rca_media_3['unique_invoice_dates'].apply(calculo_rca_media_3, args=(max_date,))
rca_media_3

,customer_id,unique_invoice_dates,recencia_media_3
0,12346,[2017-01-16 00:00:00],325.00
1,12347,"[2017-07-31 00:00:00, 2016-12-05 00:00:00, 201...",43.00
2,12348,"[2017-04-03 00:00:00, 2017-01-23 00:00:00, 201...",106.00
3,12349,[2017-11-19 00:00:00],18.00
4,12350,[2017-01-31 00:00:00],310.00
...,...,...,...
4334,18280,[2017-03-05 00:00:00],277.00
4335,18281,[2017-06-10 00:00:00],180.00
4336,18282,"[2017-08-03 00:00:00, 2017-11-30 00:00:00]",63.00
4337,18283,"[2017-04-19 00:00:00, 2017-10-25 00:00:00, 201...",5.33


RECÊNCIA média, em dias, de todas as compras:

In [23]:
# RECÊNCIA média, em dias, de todas as compras:

# Agrupando todos os chamados de cada cliente e utilizando o set para pegar apenas uma data unica, estava
# repetindo as datas, pois cada linha do dataset é um produto, se o cliente comprou mais de um produto no 
# mesmo dia resultava em várias datas iguais na coluna invoice_date com código de transação invoice_no repetido
rca_total_media = df2b.groupby('customer_id')['invoice_date'].agg(lambda x: list(set(x))).reset_index()

# Renomeando colunas
rca_total_media.columns = ['customer_id', 'unique_invoice_dates']

# Recencia Média em dias das 3 ultimas compras de cada cliente
max_date = df2b['invoice_date'].max()
rca_total_media['rencencia_media_total'] = rca_total_media['unique_invoice_dates'].apply(calculo_rca_media_total, args=(max_date,))
rca_total_media

,customer_id,unique_invoice_dates,rencencia_media_total
0,12346,[2017-01-16 00:00:00],325.00
1,12347,"[2017-07-31 00:00:00, 2016-12-05 00:00:00, 201...",52.43
2,12348,"[2017-04-03 00:00:00, 2017-01-23 00:00:00, 201...",89.50
3,12349,[2017-11-19 00:00:00],18.00
4,12350,[2017-01-31 00:00:00],310.00
...,...,...,...
4334,18280,[2017-03-05 00:00:00],277.00
4335,18281,[2017-06-10 00:00:00],180.00
4336,18282,"[2017-08-03 00:00:00, 2017-11-30 00:00:00]",63.00
4337,18283,"[2017-04-19 00:00:00, 2017-10-25 00:00:00, 201...",24.07


### Merging datasets de recencias

In [24]:
# Dropando colunas desnecessárias para o merge
rca_ultima_compra = rca_ultima_compra.drop(['ultima_compra', 'data_max'], axis=1)
rca_media_3 = rca_media_3.drop(['unique_invoice_dates'], axis=1)
rca_total_media = rca_total_media.drop(['unique_invoice_dates'], axis=1)

# Merging 
fatur_cliente = pd.merge(fatur_cliente, rca_ultima_compra, on='customer_id', how='left')
fatur_cliente = pd.merge(fatur_cliente, rca_media_3, on='customer_id', how='left')
fatur_cliente = pd.merge(fatur_cliente, rca_total_media, on='customer_id', how='left')
fatur_cliente

,customer_id,total_valor,total_transacoes,ticket_medio,recencia_ultima_compra,recencia_media_3,rencencia_media_total
0,12346,77183.60,1,77183.60,325,325.00,325.00
1,12347,4310.00,7,615.71,2,43.00,52.43
2,12348,1797.24,4,449.31,75,106.00,89.50
3,12349,1757.55,1,1757.55,18,18.00,18.00
4,12350,334.40,1,334.40,310,310.00,310.00
...,...,...,...,...,...,...,...
4334,18280,180.60,1,180.60,277,277.00,277.00
4335,18281,80.82,1,80.82,180,180.00,180.00
4336,18282,178.05,2,89.03,7,63.00,63.00
4337,18283,2045.53,16,127.85,3,5.33,24.07


In [25]:
# FAZER FEATURE DE CANCELADOS PEGAR APENAS O TOTAL DE CADA CLIENTE, PORQUE SE UM CLIENTE CANCELA MUITO QUEREMOS VER ISSO, POIS SE UM CLIENTE GOSTAR MUITO DA EMPRESA
# ELE COMPRA MAIS SEM CANCELAR





# AQUI o tópico 2 será só o feature engineering, estatística descritiva colocar no final do tópico 1.

-----------------

-----------------

-----------------

In [ ]:
# Faturamento por linha /////////// OK JA FIZ
df2['faturamento'] = df2['quantity'] * df2['unit_price']

# Monetary ## ///////////////////     VER O QUE É O GROSS REVENUE ------------------------------------////////////////////////////
df_monetary = df2[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

# Recency - Last day purchase # ////////////////   OK - --- SÓ LEMBRAR DE INVERTER OS VALORES NO LAMBDA ALI EM BAIXO
# **inverter função**
df_recency = df2[['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_recency['recency_days'] = df_recency['recency_days'].apply( lambda x: 1 / x if x != 0 else 0)
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )

# Frequency ### ---- OK JÁ FIZ, É A COLUNA total_transacoes ------------- /////////
df_freq = df2[['customer_id', 'invoice_no']].drop_duplicates().groupby( 'customer_id' ).count().reset_index()
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )

# Avg Ticket //// VERIFICAR PQ EU JÁ FIZ, MAS NÃO USEI ESSE GROSS REVENUE
df_avg_ticket = df2[['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue':'avg_ticket'} )
df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')
# print(f"{df2.isna().sum().sum()} valores NAN encontrados.")

# **3.0 Filtragem de Variáveis**

In [ ]:
df3 = df2.copy()

## 3.1 Filtragem das Colunas

In [ ]:
# Filtragem/remoção/drop de colunas com valores constantes ou 100% nulos.
cols_drop = df3.nunique()[df3.nunique() <= 1].index.tolist() # Dropando 
df3 = df3.drop(cols_drop, axis=1)

# Removido as colunas 'veiculo_alienado' e 'elegivel_revisao', pois não fornecem informações para o modelo de machine learning, 
# já que todos os valores, em ambas as colunas possuiam um único valor.


# Remoção da coluna versão, devido a feature engineering aplicada nela
df3 = df3.drop(['versao'], axis=1)


print(f"df2: {df2.shape[1]} colunas.\ndf3: {df3.shape[1]} colunas.")
print(f"{df2.shape[1] - df3.shape[1]} colunas foram removidas.")

df2: 36 colunas.
df3: 33 colunas.
3 colunas foram removidas.


## 3.2 Exportando dataset processado

In [ ]:
# df3.to_csv('../datasets/cooked_datasets/df3_processed.csv', index=False)